In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [13]:
# definding seasons thats we will scrape
years = list(range(2023, 2019, -1))
years

[2023, 2022, 2021, 2020]

In [14]:
standings_url = "https://fbref.com/en/comps/8/Champions-League-Stats#all_rank_key"

In [15]:
all_matches =[]

In [32]:
# scraping mutiple teams
for year in years:
    time.sleep(2)    
    data = requests.get(standings_url)
    time.sleep(2)
    soup = BeautifulSoup(data.text)
    time.sleep(2)
    standings_table = soup.select('table.stats_table')[8]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        time.sleep(2)
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
         #  team_data = team_data[team_data["Comp"] == "Champions League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(2)

In [34]:
len(all_matches)

128

In [35]:
match_df = pd.concat(all_matches)

In [36]:
match_df.columns = [c.lower() for c in match_df.columns]

In [37]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-07-30,20:30,DFL-Supercup,DFL-Supercup,Sat,Away,W,5.0,3.0,RB Leipzig,...,Match Report,NaN,12.0,8.0,NaN,NaN,0.0,0.0,2023,Bayern Munich
1,2022-08-05,20:30,Bundesliga,Matchweek 1,Fri,Away,W,6.0,1.0,Eint Frankfurt,...,Match Report,NaN,23.0,10.0,16.6,1.0,0.0,0.0,2023,Bayern Munich
2,2022-08-14,17:30,Bundesliga,Matchweek 2,Sun,Home,W,2.0,0.0,Wolfsburg,...,Match Report,NaN,24.0,8.0,18.4,0.0,0.0,0.0,2023,Bayern Munich
3,2022-08-21,17:30,Bundesliga,Matchweek 3,Sun,Away,W,7.0,0.0,Bochum,...,Match Report,NaN,21.0,11.0,18.3,1.0,1.0,1.0,2023,Bayern Munich
4,2022-08-27,18:30,Bundesliga,Matchweek 4,Sat,Home,D,1.0,1.0,M'Gladbach,...,Match Report,NaN,33.0,20.0,17.4,1.0,0.0,0.0,2023,Bayern Munich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2020-02-13,20:30,First Division A,Regular season,Thu,Away,D,1.0,1.0,Antwerp,...,Match Report,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2020,Genk
31,2020-02-16,18:00,First Division A,Regular season,Sun,Home,L,1.0,3.0,Standard Liège,...,Match Report,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2020,Genk
32,2020-02-21,20:00,First Division A,Regular season,Fri,Away,W,1.0,0.0,Kortrijk,...,Match Report,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2020,Genk
33,2020-03-01,14:30,First Division A,Regular season,Sun,Home,L,1.0,2.0,Club Brugge,...,Match Report,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2020,Genk


In [38]:
match_df.to_csv("matches.csv")